In [2]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt

In [3]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e' or base == 'Delta':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s])}

## Target function

we want to minimize the effective price, given a set of exponents for the buying
and selling asset each.

In [4]:
def deltaBySpot_(s,v, b, w, j, e, Delta, a):
  return simplify((s - (v + b) * w) / w)

deltaBySpot = {asset: deltaBySpot_(*all_symbols[asset]) for asset in assets}
deltaBySpot['b']

-b_b + s_b/w_b - v_b

In [5]:
def spotByExp_(s, v, b, w, j, e, Delta, a):
  return simplify(a * ((1 + 1/j) ** e)
)
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

a_b*((j_b + 1)/j_b)**e_b

In [6]:
def deltaByExp_(asset, s, v, b, w, j, e, Delta, a):
    return simplify(deltaBySpot[asset].subs(s, spotByExp[asset]))

deltaByExp = {asset: deltaByExp_(asset, *all_symbols[asset]) for asset in assets}
deltaByExp['b']

a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b - b_b - v_b

In [7]:
eff = Delta_s / -Delta_b;
eff;

-Delta_s/Delta_b

In [8]:
deltaByExp_b = deltaByExp["b"];
deltaByExp_s = deltaByExp["s"];
effByExps = eff.subs({ Delta_b: deltaByExp_b, Delta_s: deltaByExp_s });
effByExps;

-(a_s*j_s**(-e_s)*(j_s + 1)**e_s/w_s - b_s - v_s)/(a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b - b_b - v_b)

In [10]:
print(effByExps);

-(a_s*j_s**(-e_s)*(j_s + 1)**e_s/w_s - b_s - v_s)/(a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b - b_b - v_b)


### convexity of the target function

- deltaBySpot is linear
- spotByExp is convex
- deltaByExp is a composition of the two

=> deltaByExp is convex

- negation flips convexity/concavity
- deltaBuyingByExp is negative => -deltaBuyingByExp is positive
- inversion of a positive function flips convexity/concavity => 1 /
  (-deltaBuyingByExp) is convex
- multiplying two nonnegative functions with different variables, that are each
  convex and independent of the other's variable, results in a convex function

==> effectivePrice = deltaSellingByExp / (-deltaBuyingByExp) is convex. qed

## inequality-constraints

- the exponents must adhere to their upper (buying) resp. lower (selling) bounds
  given by our equation
- need to buy and sell minimum amounts (TODO)
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not
  exceed expLimit (TODO)

In [8]:
def expBound_(asset, s, v, b, w, j, e, Delta, a):
    e_bound = simplify(log(w * (v + b) /a, 1 + 1/j)) # constant wrt e
    if asset == 'b':
        return e - e_bound # <=! 0 (upper bound)
    else:
        return e_bound - e # <=! 0 (lower bound)

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

e_b - log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b)))

In [9]:
expBound["s"];

-e_s + log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s)))

In [10]:
balanceBound = -Delta_b - b_b # <=! 0
balanceBound = balanceBound.subs({Delta_b: deltaByExp_b})
balanceBound

-a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b + v_b

In [11]:
max_s, max_I = symbols('max_s, max_I', positive=True, integer=True)

maxSelling = Delta_s - max_s # <=! 0
maxSelling = maxSelling.subs({Delta_s: deltaByExp_s})
maxSelling

a_s*j_s**(-e_s)*(j_s + 1)**e_s/w_s - b_s - max_s - v_s

In [12]:
def maxSpot_(asset, s, v, b, w, j, e, Delta, a):
  return spotByExp[asset] - max_I # <=! 0

maxSpot = {asset: maxSpot_(asset, *all_symbols[asset]) for asset in assets}
maxSpot['b']

a_b*((j_b + 1)/j_b)**e_b - max_I

## Lagrangian

In [13]:
l1, l2, l3, l4, l5, l6 = symbols('lambda_1 lambda_2 lambda_3 lambda_4 lambda_5 lambda_6', positive=True)

L = effByExps + l1 * expBound['b'] + l2 * expBound['s']# + l3 * balanceBound + l4 * maxSelling + l5 * maxSpot['b'] + l6 * maxSpot['s']
L

lambda_1*(e_b - log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b)))) + lambda_2*(-e_s + log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s)))) - (a_s*j_s**(-e_s)*(j_s + 1)**e_s/w_s - b_s - v_s)/(a_b*j_b**(-e_b)*(j_b + 1)**e_b/w_b - b_b - v_b)

In [14]:
# Partial derivatives
dL_des = L.diff(e_s)
dL_deb = L.diff(e_b)
dL_dl1 = L.diff(l1)
dL_dl2 = L.diff(l2)
# dL_dl3 = L.diff(l3)
# dL_dl4 = L.diff(l4)
# dL_dl5 = L.diff(l5)
# dL_dl6 = L.diff(l6)

# Set them to zero
stationary_conditions = [
    Eq(dL_des, 0),
    Eq(dL_deb, 0),
    Eq(dL_dl1, 0),
    Eq(dL_dl2, 0),
    # Eq(dL_dl3, 0),
    # Eq(dL_dl4, 0),
    # Eq(dL_dl5, 0),
    # Eq(dL_dl6, 0),
]

In [16]:
solutions = solve(stationary_conditions, (e_s, e_b, l1, l2))#, l3, l4, l5, l6))

solutions = solutions[0]
# e_s_sol, e_b_sol, l1_sol, l2_sol, l3_sol, l4_sol, l5_sol, l6_sol = solutions
e_s_sol, e_b_sol, l1_sol, l2_sol = solutions
solutions

(log((a_s/(w_s*(b_s + v_s)))**(1/(log(j_s) - log(j_s + 1)))),
 log((a_b/(w_b*(b_b + v_b)))**(1/(log(j_b) - log(j_b + 1)))),
 log((j_b**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*(j_b + 1)**(j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))*w_s*(b_s + v_s)))**(a_b*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_b*(j_b + 1))**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))/w_s)*(j_b**(j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))*w_s*(b_s + v_s))*(j_b + 1)**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))))**(-a_b*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_b*(j_b + 1))**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b 

In [17]:
e_s_sol;

log((a_s/(w_s*(b_s + v_s)))**(1/(log(j_s) - log(j_s + 1))))

In [18]:
e_b_sol;

log((a_b/(w_b*(b_b + v_b)))**(1/(log(j_b) - log(j_b + 1))))

Note that the e_i only depend on variables for i, therefore can be optimized
independently.

In [189]:
simplify(deltaByExp_b.subs({ e_b: e_b_sol }));

a_b*j_b**(-log(a_b**(1/(log(j_b) - log(j_b + 1)))*(b_b*w_b + v_b*w_b)**(-1/(log(j_b) - log(j_b + 1)))))*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(b_b*w_b + v_b*w_b)**(-1/(log(j_b) - log(j_b + 1))))/w_b - b_b - v_b

In [190]:
simplify(deltaByExp_s.subs({ e_s: e_s_sol }));

a_s*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(b_s*w_s + v_s*w_s)**(-1/(log(j_s) - log(j_s + 1)))))*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(b_s*w_s + v_s*w_s)**(-1/(log(j_s) - log(j_s + 1))))/w_s - b_s - v_s

In [19]:
l1_sol;

log((j_b**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*(j_b + 1)**(j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))*w_s*(b_s + v_s)))**(a_b*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_b*(j_b + 1))**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))/w_s)*(j_b**(j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))*w_s*(b_s + v_s))*(j_b + 1)**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))))**(-a_b*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_b*(j_b + 1))**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))/w_s))/(-a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1

In [20]:
l2_sol;

log(j_s**(-a_s*j_b**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))/w_s)*(j_s + 1)**(a_s*j_b**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))/w_s))/(-a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1)))) + b_b*j_b**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))*w_b + j_b**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))*v_b*w_b)

In [21]:
#l3_sol;

In [22]:
#l4_sol;

In [23]:
#l5_sol;

In [24]:
#l6_sol;

In [25]:
for sol in solutions:
    print(sol.free_symbols)

{j_s, v_s, b_s, w_s, a_s}
{b_b, w_b, v_b, j_b, a_b}
{j_s, v_s, b_b, a_b, j_b, b_s, w_s, w_b, v_b, a_s}
{j_s, v_s, b_b, a_b, j_b, b_s, w_s, w_b, v_b, a_s}


Analysing l1_sol:

In [26]:
print(l1_sol);

log((j_b**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*(j_b + 1)**(j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))*w_s*(b_s + v_s)))**(a_b*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_b*(j_b + 1))**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))/w_s)*(j_b**(j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))*w_s*(b_s + v_s))*(j_b + 1)**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))))**(-a_b*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_b*(j_b + 1))**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))/w_s))/(-a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1

In [49]:
u, v, p, q, r = symbols('u v p q r')

subs_dict = {
    log(a_s**(1/(log(j_s) - log(j_s + 1))) * (w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))): u,
    log(a_b**(1/(log(j_b) - log(j_b + 1))) * (w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1)))): v,
    j_s**log(a_s**(1/(log(j_s) - log(j_s + 1))) * (w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))) * w_s*(b_s + v_s): p,
    j_b**log(a_b**(1/(log(j_b) - log(j_b + 1))) * (w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1)))) * w_b*(b_b + v_b): q
}

subs_dict2 = {
    a_b*j_s**(-u)*w_b*(j_b*(j_b + 1))**v/w_s: r,
}

# Apply substitutions to l1_sol
l1_sol_subs = l1_sol.subs(subs_dict)
l1_sol_subs = l1_sol_subs.subs(subs_dict2)
l1_sol_subs


log((j_b**p*(j_b + 1)**(a_s*(j_s + 1)**u))**(-r)*(j_b**(a_s*(j_s + 1)**u)*(j_b + 1)**p)**r)/(-a_b*(j_b + 1)**v + b_b*j_b**v*w_b + j_b**v*v_b*w_b)**2

In [50]:
print(l1_sol_subs);

log((j_b**p*(j_b + 1)**(a_s*(j_s + 1)**u))**(-r)*(j_b**(a_s*(j_s + 1)**u)*(j_b + 1)**p)**r)/(-a_b*(j_b + 1)**v + b_b*j_b**v*w_b + j_b**v*v_b*w_b)**2


numerator:


In [52]:
log(
  (j_b ** p * (j_b + 1) ** (a_s * (j_s + 1) ** u)) ** (-r) *
    (j_b ** (a_s * (j_s + 1) ** u) * (j_b + 1) ** p) ** r,
);

log((j_b**p*(j_b + 1)**(a_s*(j_s + 1)**u))**(-r)*(j_b**(a_s*(j_s + 1)**u)*(j_b + 1)**p)**r)

In [53]:
(j_b ** p * (j_b + 1) ** (a_s * (j_s + 1) ** u)) ** (-r) *
  (j_b ** (a_s * (j_s + 1) ** u) * (j_b + 1) ** p) ** r;

(j_b**p*(j_b + 1)**(a_s*(j_s + 1)**u))**(-r)*(j_b**(a_s*(j_s + 1)**u)*(j_b + 1)**p)**r

In [57]:
simplify(
  ((j_b ** (a_s * (j_s + 1) ** u) * (j_b + 1) ** p) /
    (j_b ** p * (j_b + 1) ** (a_s * (j_s + 1) ** u))) ** r,
);

(j_b**(a_s*(j_s + 1)**u - p)*(j_b + 1)**(-a_s*(j_s + 1)**u + p))**r

In [58]:
print(
  simplify(
    ((j_b ** (a_s * (j_s + 1) ** u) * (j_b + 1) ** p) /
      (j_b ** p * (j_b + 1) ** (a_s * (j_s + 1) ** u))) ** r,
  ),
);

(j_b**(a_s*(j_s + 1)**u - p)*(j_b + 1)**(-a_s*(j_s + 1)**u + p))**r


In [60]:
((j_b + 1) / j_b) ** ((p - a_s * (j_s + 1) ** u) * r);

((j_b + 1)/j_b)**(r*(-a_s*(j_s + 1)**u + p))

In [61]:
simplify(((j_b + 1) / j_b) ** ((p - a_s * (j_s + 1) ** u) * r));

(j_b/(j_b + 1))**(r*(a_s*(j_s + 1)**u - p))

Analysing l2_sol:

In [27]:
# Define new symbols for substitutions
u, v, p, q, r = symbols('u v p q r')

# Define the substitutions
subs_dict = {
    log(a_s ** (1 / (log(j_s) - log(j_s + 1))) * ((b_s + v_s) * w_s) ** (-1 / (log(j_s) - log(j_s + 1)))): u,
    log(a_b ** (1 / (log(j_b) - log(j_b + 1))) * ((b_b + v_b) * w_b) ** (-1 / (log(j_b) - log(j_b + 1)))): v,
    j_s ** (a_s * j_b ** v) * j_s ** (-u): p,
    (j_s + 1) ** u: q,
    # Add other substitutions as necessary
}

subs_dict2 = {
    a_s*j_b**v*j_s**(-u)*q*w_b/w_s: r
}

# Substitute in the expression for l2
l2_substituted = l2_sol.subs(subs_dict)
l2_substituted = l2_substituted.subs(subs_dict2)

l2_substituted

log(j_s**(-r)*(j_s + 1)**r)/(-a_b*(j_b + 1)**v + b_b*j_b**v*w_b + j_b**v*v_b*w_b)

we need to check if this is positive, negative, or zero.

Starting with the numerator:

In [28]:
print(list(subs_dict2.keys())[0]) # r
print(list(subs_dict.keys())[3]) # q

a_s*j_b**v*j_s**(-u)*q*w_b/w_s
(j_s + 1)**u


factors of r:

- a_s -> positive
- j_b**v -> positive
- j_s**(-u) -> positive
- q = (j_s + 1)**u -> positive
- w_b -> positive
- w_s -> positive

=> r is always positive

=> ((j_s + 1) / j_s)**r is always > 1

=> log(((j_s + 1) / j_s)**r) is always positive

=> numerator is always positive

Next the denominator, which ideally is always positive as well:

In [29]:
print(l2_substituted);

log(j_s**(-r)*(j_s + 1)**r)/(-a_b*(j_b + 1)**v + b_b*j_b**v*w_b + j_b**v*v_b*w_b)


In [30]:
l2_denom = -a_b * (j_b + 1) ** v + b_b * j_b ** v * w_b + j_b ** v * v_b * w_b;
Lt(0, l2_denom);

0 < -a_b*(j_b + 1)**v + b_b*j_b**v*w_b + j_b**v*v_b*w_b

In [31]:
l2_denom_eq = Lt(
  a_b * (j_b + 1) ** v,
  b_b * j_b ** v * w_b + j_b ** v * v_b * w_b,
);
l2_denom_eq;

a_b*(j_b + 1)**v < b_b*j_b**v*w_b + j_b**v*v_b*w_b

In [32]:
l2_denom_eq2 = Lt(((j_b + 1) / j_b) ** v, b_b * w_b / a_b + v_b * w_b / a_b);
l2_denom_eq2;

((j_b + 1)/j_b)**v < b_b*w_b/a_b + v_b*w_b/a_b

In [33]:
#minAnchorPrices;
Le((v_b * w_b * j_b) / (j_b + 1), a_b);

j_b*v_b*w_b/(j_b + 1) <= a_b

In [34]:
Le(j_b / (j_b + 1), a_b / (v_b * w_b));

j_b/(j_b + 1) <= a_b/(v_b*w_b)

In [35]:
Le((v_b * w_b) / a_b, (j_b + 1) / j_b);

v_b*w_b/a_b <= (j_b + 1)/j_b

### Complementary Slackness

for each inequality constraint, it should hold that lambda_i * g_i(x) = 0

In [67]:
g1 = expBound["b"].subs({ e_b: e_b_sol });
g1;

-log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b))) + log((a_b/(w_b*(b_b + v_b)))**(1/(log(j_b) - log(j_b + 1))))

In [78]:
slack1 = g1 * l1_sol;
slack1;

(-log((w_b*(b_b + v_b)/a_b)**(1/log((j_b + 1)/j_b))) + log((a_b/(w_b*(b_b + v_b)))**(1/(log(j_b) - log(j_b + 1)))))*log((j_b**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*(j_b + 1)**(j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))*w_s*(b_s + v_s)))**(a_b*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_b*(j_b + 1))**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))/w_s)*(j_b**(j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))*w_s*(b_s + v_s))*(j_b + 1)**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))))**(-a_b*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_b*(j_b + 1))**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**

In [79]:
slack1.func;

sympy.core.mul.Mul

In [77]:
slack1_ = simplify(slack1);
slack1_;

-log((j_b**(a_s*(j_s + 1)**log(a_s**(1/log(j_s/(j_s + 1)))*(w_s*(b_s + v_s))**(-1/log(j_s/(j_s + 1)))) - j_s**log(a_s**(1/log(j_s/(j_s + 1)))*(w_s*(b_s + v_s))**(-1/log(j_s/(j_s + 1))))*w_s*(b_s + v_s))*(j_b + 1)**(-a_s*(j_s + 1)**log(a_s**(1/log(j_s/(j_s + 1)))*(w_s*(b_s + v_s))**(-1/log(j_s/(j_s + 1)))) + j_s**log(a_s**(1/log(j_s/(j_s + 1)))*(w_s*(b_s + v_s))**(-1/log(j_s/(j_s + 1))))*w_s*(b_s + v_s)))**log((a_b**(-1/log(j_b/(j_b + 1)) - 1/log((j_b + 1)/j_b))*(w_b*(b_b + v_b))**(1/log(j_b/(j_b + 1)) + 1/log((j_b + 1)/j_b)))**(a_b*j_s**(-log(a_s**(1/log(j_s/(j_s + 1)))*(w_s*(b_s + v_s))**(-1/log(j_s/(j_s + 1)))))*w_b*(j_b*(j_b + 1))**log(a_b**(1/log(j_b/(j_b + 1)))*(w_b*(b_b + v_b))**(-1/log(j_b/(j_b + 1))))/w_s)))/(-a_b*(j_b + 1)**log(a_b**(1/log(j_b/(j_b + 1)))*(w_b*(b_b + v_b))**(-1/log(j_b/(j_b + 1)))) + b_b*j_b**log(a_b**(1/log(j_b/(j_b + 1)))*(w_b*(b_b + v_b))**(-1/log(j_b/(j_b + 1))))*w_b + j_b**log(a_b**(1/log(j_b/(j_b + 1)))*(w_b*(b_b + v_b))**(-1/log(j_b/(j_b + 1))))*v_b*w_b

In [120]:
# Define new symbols for substitutions
u, v, p, q, r = symbols('u v p q r')

# Define the substitutions
subs_dict = {
    1/log(j_b/(j_b + 1)): u,
    1/log(j_s/(j_s + 1)): v,
    (a_s*(j_s + 1)**log(a_s**v*(w_s*(b_s + v_s))**(-v)) - j_s**log(a_s**v*(w_s*(b_s + v_s))**(-v))*w_s*(b_s + v_s)): p,
    # Add other substitutions as necessary
}
slack1_ = slack1_.subs(subs_dict)
slack1_

-log((j_b**p*(j_b + 1)**(-p))**log((a_b**(-u - 1/log((j_b + 1)/j_b))*(w_b*(b_b + v_b))**(u + 1/log((j_b + 1)/j_b)))**(a_b*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_b*(j_b + 1))**log(a_b**u*(w_b*(b_b + v_b))**(-u))/w_s)))/(-a_b*(j_b + 1)**log(a_b**u*(w_b*(b_b + v_b))**(-u)) + b_b*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*w_b + j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*v_b*w_b)**2

In [121]:
slack1_.func;

sympy.core.mul.Mul

In [122]:
len(slack1_.args);

3

In [123]:
slack1_.args[0];

-1

In [124]:
slack1_.args[1];

(-a_b*(j_b + 1)**log(a_b**u*(w_b*(b_b + v_b))**(-u)) + b_b*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*w_b + j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*v_b*w_b)**(-2)

-> denominator should not be 0

In [176]:
denbase = slack1_.args[1].args[0];
denbase;

-a_b*(j_b + 1)**log(a_b**u*(w_b*(b_b + v_b))**(-u)) + b_b*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*w_b + j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*v_b*w_b

In [180]:
denbase0 = solve(Eq(denbase, 0));
len(denbase0);

1

In [185]:
denbase0[0];

{j_b: a_b**(1/log(a_b**u*(w_b*(b_b + v_b))**(-u)))/(-a_b**(1/log(a_b**u*(w_b*(b_b + v_b))**(-u))) + (w_b*(b_b + v_b))**(1/log(a_b**u*(w_b*(b_b + v_b))**(-u))))}

In [187]:
list(denbase0[0].values())[0];

a_b**(1/log(a_b**u*(w_b*(b_b + v_b))**(-u)))/(-a_b**(1/log(a_b**u*(w_b*(b_b + v_b))**(-u))) + (w_b*(b_b + v_b))**(1/log(a_b**u*(w_b*(b_b + v_b))**(-u))))

In [125]:
slack1_.args[2];

log((j_b**p*(j_b + 1)**(-p))**log((a_b**(-u - 1/log((j_b + 1)/j_b))*(w_b*(b_b + v_b))**(u + 1/log((j_b + 1)/j_b)))**(a_b*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_b*(j_b + 1))**log(a_b**u*(w_b*(b_b + v_b))**(-u))/w_s)))

-> this last factor is the only one that can reach 0

In [126]:
slack1_.args[2].func;

log

-> log is 0 if the argument is 1

In [127]:
slack1_.args[2].args[0];

(j_b**p*(j_b + 1)**(-p))**log((a_b**(-u - 1/log((j_b + 1)/j_b))*(w_b*(b_b + v_b))**(u + 1/log((j_b + 1)/j_b)))**(a_b*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_b*(j_b + 1))**log(a_b**u*(w_b*(b_b + v_b))**(-u))/w_s))

In [128]:
slack1_.args[2].args[0].func;

sympy.core.power.Pow

-> power is 1 if the base is 1 or the exponent is 0

In [133]:
exp = slack1_.args[2].args[0].args[1];
exp;

log((a_b**(-u - 1/log((j_b + 1)/j_b))*(w_b*(b_b + v_b))**(u + 1/log((j_b + 1)/j_b)))**(a_b*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_b*(j_b + 1))**log(a_b**u*(w_b*(b_b + v_b))**(-u))/w_s))

In [134]:
exp.func;

log

-> log is 0 if the argument is 1

In [135]:
exp.args[0];

(a_b**(-u - 1/log((j_b + 1)/j_b))*(w_b*(b_b + v_b))**(u + 1/log((j_b + 1)/j_b)))**(a_b*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_b*(j_b + 1))**log(a_b**u*(w_b*(b_b + v_b))**(-u))/w_s)

In [136]:
exp.args[0].func;

sympy.core.power.Pow

-> power is 1 if the base is 1 or the exponent is 0

In [140]:
base2 = exp.args[0].args[0];
base2;

a_b**(-u - 1/log((j_b + 1)/j_b))*(w_b*(b_b + v_b))**(u + 1/log((j_b + 1)/j_b))

-> base2 is 1 if that u + 1/log((j_b + 1) / j_b) exponent is 0

u = 1/log(j_b/(j_b + 1))

In [143]:
exp3 = 1 / log(j_b / (j_b + 1)) + 1 / log((j_b + 1) / j_b);
exp3;

1/log(j_b/(j_b + 1)) + 1/log((j_b + 1)/j_b)

In [144]:
simplify(exp3);

0

qed

TODO: check if top-level denominator is zero

In [145]:
g2 = expBound["s"].subs({ e_s: e_s_sol });
g2;

log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s))) - log((a_s/(w_s*(b_s + v_s)))**(1/(log(j_s) - log(j_s + 1))))

In [146]:
slack2 = g2 * l2_sol;
slack2;

(log((w_s*(b_s + v_s)/a_s)**(1/log((j_s + 1)/j_s))) - log((a_s/(w_s*(b_s + v_s)))**(1/(log(j_s) - log(j_s + 1)))))*log(j_s**(-a_s*j_b**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))/w_s)*(j_s + 1)**(a_s*j_b**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))*j_s**(-log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1)))))*w_b*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))*(w_s*(b_s + v_s))**(-1/(log(j_s) - log(j_s + 1))))/w_s))/(-a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1)))) + b_b*j_b**log(a_b**(1/(log(j_b) - log(j_b + 1)))*(w_b*(b_b + v_b))**(-1/(log(j_b) - log(j_b + 1))))*w_b + j_b**log(a_b**(1/(log(j_b) - log(

In [159]:
slack2_ = simplify(slack2);
slack2_ = slack2_.subs(subs_dict);
slack2_;

log((j_s**(-a_s*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_s + 1)**log(a_s**v*(w_s*(b_s + v_s))**(-v))/w_s)*(j_s + 1)**(a_s*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_s + 1)**log(a_s**v*(w_s*(b_s + v_s))**(-v))/w_s))**log(a_s**(-v - 1/log((j_s + 1)/j_s))*(w_s*(b_s + v_s))**(v + 1/log((j_s + 1)/j_s))))/(-a_b*(j_b + 1)**log(a_b**u*(w_b*(b_b + v_b))**(-u)) + b_b*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*w_b + j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*v_b*w_b)

In [160]:
slack2_.func;

sympy.core.mul.Mul

In [161]:
len(slack2_.args);

2

In [162]:
slack2_.args[0];

1/(-a_b*(j_b + 1)**log(a_b**u*(w_b*(b_b + v_b))**(-u)) + b_b*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*w_b + j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*v_b*w_b)

In [163]:
slack2_.args[1];

log((j_s**(-a_s*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_s + 1)**log(a_s**v*(w_s*(b_s + v_s))**(-v))/w_s)*(j_s + 1)**(a_s*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_s + 1)**log(a_s**v*(w_s*(b_s + v_s))**(-v))/w_s))**log(a_s**(-v - 1/log((j_s + 1)/j_s))*(w_s*(b_s + v_s))**(v + 1/log((j_s + 1)/j_s))))

-> this factor is the only one that can reach 0

In [164]:
slack2_.args[1].func;

log

-> logarithm is 0 if the argument is 1

In [165]:
slack2_.args[1].args[0];

(j_s**(-a_s*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_s + 1)**log(a_s**v*(w_s*(b_s + v_s))**(-v))/w_s)*(j_s + 1)**(a_s*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_s + 1)**log(a_s**v*(w_s*(b_s + v_s))**(-v))/w_s))**log(a_s**(-v - 1/log((j_s + 1)/j_s))*(w_s*(b_s + v_s))**(v + 1/log((j_s + 1)/j_s)))

In [166]:
slack2_.args[1].args[0].func;

sympy.core.power.Pow

-> power is 1 if the base is 1 or the exponent is 0

In [167]:
base = slack2_.args[1].args[0].args[0];
base;

j_s**(-a_s*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_s + 1)**log(a_s**v*(w_s*(b_s + v_s))**(-v))/w_s)*(j_s + 1)**(a_s*j_b**log(a_b**u*(w_b*(b_b + v_b))**(-u))*j_s**(-log(a_s**v*(w_s*(b_s + v_s))**(-v)))*w_b*(j_s + 1)**log(a_s**v*(w_s*(b_s + v_s))**(-v))/w_s)

In [168]:
exp = slack2_.args[1].args[0].args[1];
exp;

log(a_s**(-v - 1/log((j_s + 1)/j_s))*(w_s*(b_s + v_s))**(v + 1/log((j_s + 1)/j_s)))

In [169]:
exp.func;

log

-> logarithm is 0 if the argument is 1

In [170]:
exp.args[0];

a_s**(-v - 1/log((j_s + 1)/j_s))*(w_s*(b_s + v_s))**(v + 1/log((j_s + 1)/j_s))

-> this is 1 if that v + 1/log((j_s + 1) /j_s) is 0

v = 1/log(j_s/(j_s + 1))

In [171]:
exp2 = 1 / log(j_s / (j_s + 1)) + 1 / log((j_s + 1) / j_s);
exp2;

1/log(j_s/(j_s + 1)) + 1/log((j_s + 1)/j_s)

In [172]:
simplify(exp2);

0

qed

TODO: check if denominator is zero